In [1]:
import numpy as np
import tqdm 
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import colors
import matplotlib as mpl 
import astropy.units as u
from astropy.constants import G, M_sun
import sys
import os

sys.path.append('../my_funcs/')
from pipeline_main import pipeline, dist, calc_ang


In [2]:
data = pipeline(snap = 400, run = 'sink122core02', sink_id = 122)

Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 5026/5026 [00:16<00:00, 313.14it/s]


In [3]:
data.recalc_L_old(); data.L, data.total_L

AttributeError: 'pipeline' object has no attribute 'recalc_L_old'

In [ ]:
data.recalc_L(); data.L, data.total_L

Converged mean angular momentum vector after 0 iteration(s)


(array([-0.91798873,  0.38220767, -0.10589616]), 4.686650078481343e-09)

In [ ]:
L_new = np.zeros(3)
height = 15; radius = 150; err_deg = 5; verbose = 1
height /= data.au_length; radius /= data.au_length

w = np.array([p.level for p in data.sn.patches]).argsort()[::-1]
sorted_patches = [data.sn.patches[w[i]] for i in range(len(data.sn.patches))]

for p in sorted_patches:
    nbors = [data.sn.patchid[i] for i in p.nbor_ids if i in data.sn.patchid]
    children = [ n for n in nbors if n.level == p.level + 1]
    leafs = [n for n in children if ((n.position - p.position)**2).sum() < ((p.size)**2).sum()/12]   
    if len(leafs) == 8: continue

    to_extract = (p.cyl_R < radius) & ((abs(p.cyl_z) < height) | (abs(p.cyl_z / p.cyl_R) < 0.3))
    for lp in leafs: 
        leaf_extent = np.vstack((lp.position - 0.5 * lp.size, lp.position + 0.5 * lp.size)).T
        covered_bool = ~np.all((p.xyz > leaf_extent[:, 0, None, None, None]) & (p.xyz < leaf_extent[:, 1, None, None, None]), axis=0)
        to_extract *= covered_bool 
   
    L_patch = np.cross(p.rel_xyz, p.vel_xyz * p.m , axisa=0, axisb=0, axisc=0)
    L_new += np.array([np.sum(L_patch[axis, to_extract]) for axis in range(3)])


In [ ]:
L_new / np.linalg.norm(L_new)

array([-0.90232398,  0.41016879, -0.1325632 ])

In [ ]:

#### OLD RECALC_FUNCTION FOUND HERE #### UPDATED VERSION IS FOUND IN PIPELINE

def recalc_L_old(self, height = 10, radius = 100, err_deg = 5, verbose = 1):
    if not self.cyl_calculated: self.calc_cyl()
    height /= self.au_length; radius /= self.au_length

    def recalc():
        L_new = np.zeros(3)
        for p in self.sn.patches:
            idx = np.nonzero((p.cyl_R < radius) & ((abs(p.cyl_z) < height) | (abs(p.cyl_z / p.cyl_R) < 0.3)))
            L_patch = np.cross(p.rel_xyz, p.vel_xyz * p.m , axisa=0, axisb=0, axisc=0)
            L_new += np.array([np.sum(L_patch[axis][idx]) for axis in range(3)])
        return L_new 
    L_new =  recalc()
    L_i = 0
    while calc_ang(self.L, L_new) > err_deg:
        self.L = L_new / np.linalg.norm(L_new); self.total_L = np.linalg.norm(L_new)
        self.calc_cyl()
        L_new = recalc()
        L_i += 1
        if L_i > 20: break
    if verbose != 0: print(f'Converged mean angular momentum vector after {L_i} iteration(s)')

In [5]:
for p in data.sn.patches:
    if p.id == 1853: break

In [7]:
p.__dir__()

['id',
 'memmap',
 'ioformat',
 'iout',
 'time',
 'ntotal',
 'box',
 'li',
 'ui',
 'ng',
 'gn',
 'n',
 'nv',
 'mv',
 'nt',
 'gamma',
 'eos_name',
 'opacity',
 'periodic',
 'guard_zones',
 'time_derivs',
 'no_mans_land',
 'omp_nthreads',
 'mpi_size',
 'mesh_type',
 'mpi_dims',
 'refine_ratio',
 'origin',
 'position',
 'size',
 'level',
 'dtime',
 'istep',
 'ds',
 'ncell',
 'velocity',
 'kind',
 'etype',
 'record',
 'rank',
 'centre_nat',
 'llc_nat',
 'erot1',
 'erot2',
 'erot3',
 'parent_id',
 'nbor_ids',
 'idx',
 'extent',
 'llc_cart',
 'filename',
 'offset',
 'ip',
 'var',
 'aux',
 'data',
 'keys',
 'all_keys',
 'x',
 'y',
 'z',
 'xi',
 'yi',
 'zi',
 'xs',
 'ys',
 'zs',
 'xyz',
 'xyzi',
 'geometric_factors',
 'parent',
 'rel_ppos',
 'rel_xyz',
 'dist_xyz',
 'vel_xyz',
 'vrel',
 'm',
 'γ',
 'P',
 '__module__',
 '__init__',
 'indices',
 'contains',
 '_h',
 'cache',
 'plane',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__